In [1]:
# import module list is independent on respective file.

import numpy as np
import pandas as pd
%matplotlib inline
import matplotlib.pyplot as plt
import sys
import yfinance as yf

from datetime import datetime as dt
from datetime import date as date
from datetime import timedelta
from dateutil.relativedelta import relativedelta

import seaborn as sns
from scipy.stats import mstats

from tqdm.notebook import tqdm

import japanize_matplotlib



In [4]:
import calendar
from datetime import date 
def get_nth_week(day):
    return (day - 1) // 7 + 1

def get_nth_dow(year, month, day):
    return get_nth_week(day), calendar.weekday(year, month, day)

def get_day_of_nth_dow(year, month, nth, dow):
    '''dow: Monday(0) - Sunday(6)'''
    if nth < 1 or dow < 0 or dow > 6:
        return None

    first_dow, n = calendar.monthrange(year, month)
    day = 7 * (nth - 1) + (dow - first_dow) % 7 + 1

    return day if day <= n else None
def get_nth_dow_datetime_dt(dt):
    return get_nth_week(dt.day), dt.weekday()
def get_nth_dow_datetime(year, month, day):
    return get_nth_week(day), date(year, month, day).weekday()

def get_date_of_nth_dow(year, month, nth, dow):
    day = get_day_of_nth_dow(year, month, nth, dow)
    return date(year, month, day) if day else None

# Kerasを使って　分類モデルを作ろう!

In [7]:
import warnings
warnings.simplefilter('ignore')

from sklearn.metrics import matthews_corrcoef

from scipy.stats import mstats
import japanize_matplotlib
from tqdm.notebook import tqdm

from keras.models import Sequential
from keras.layers import LSTM, Dense, Embedding, Dropout

In [8]:
def create_feature(data, window_size):
    window_data = []
    X, Y = [], []
    
    if len(data) - window_size >= 1:
        
        for i in range(len(data)-window_size):
            window = data[i:i+window_size]

            X.append(window)

            Y.append(data[i + window_size - 1])
            
            #window_data.append(window)
    else:
        for i in range(len(data)-window_size+1):
            window = data[i:i+window_size]
            #window_data.append(window)
            
    
            X.append(window)

            Y.append(data[i + window_size - 1])
    
    
    
    # [[], []. []......]
    print(np.array(X))
    return (np.array(X).reshape(-1, window_size, 1), np.array(Y))

In [ ]:
'''
def lstm_comp(df):
  # 入力層/中間層/出力層のネットワーク構築
  model = Sequential()
  model.add(LSTM(256, activation='relu', batch_input_shape=(None, df.shape[1], df.shape[2])))
  model.add(Dropout(0.2))
  model.add(Dense(256, activation='relu'))
  model.add(Dropout(0.2))
  model.add(Dense(1, activation='sigmoid'))

  # ネットワークのコンパイル
  model.compile(loss='binary_crossentropy', optimizer='adam', metrics=["accuracy"])
  return model

'''

In [95]:
df.shape

(115, 38)

In [96]:
def lstm_pred(df):
    model = Sequential()

    length = df.shape[1]#X_train.shape[1]
    hidden_layers = 128
    in_out_neurons = 1
    length_of_sequences = df.shape[0]
    #model.add(Embedding(input_dim=length, output_dim=256, input_length=5))
    model.add(LSTM(hidden_layers,
                   #batch_size=32,
                   input_shape=(length_of_sequences, in_out_neurons),
                   return_sequences=False,
                 stateful=False))
    model.add(Dropout(0.2))
    model.add(Dense(hidden_layers, activation='relu'))
    model.add(Dropout(0.2))
      
    
    # 1 or 9
    model.add(Dense(1, activation='softmax'))

    model.compile(loss='mean_squared_error', optimizer='adam')
    return model

model.summary()

Model: "sequential_1813"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━┓
┃ Layer (type)                       ┃ Output Shape                  ┃     Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━┩
│ lstm_1809 (LSTM)                   │ (None, 128)                   │      66,560 │
├────────────────────────────────────┼───────────────────────────────┼─────────────┤
│ dropout_1804 (Dropout)             │ (None, 128)                   │           0 │
├────────────────────────────────────┼───────────────────────────────┼─────────────┤
│ dense_1804 (Dense)                 │ (None, 1)                     │         129 │
└────────────────────────────────────┴───────────────────────────────┴─────────────┘

 Total params: 200,069 (781.52 KB)

 Trainable params: 66,689 (260.50 KB)

 Non-trainable params: 0 (0.00 B)

 Optimizer params: 133,380 (521.02 KB)

In [20]:
from tensorflow.keras.callbacks import EarlyStopping
callbacks = [EarlyStopping(monitor='val_loss',
                           patience=5, # ここで指定したエポック数の間改善がないと停止
                           verbose=1,
                           mode='max')
            ]
import numpy as np
from sklearn.model_selection import train_test_split



In [36]:
cols = data.columns

In [87]:
folds = TimeSeriesSplit(5)
for i in folds.split(df):
    print(i)
    

(array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16,
       17, 18, 19]), array([20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36,
       37, 38]))
(array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16,
       17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33,
       34, 35, 36, 37, 38]), array([39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55,
       56, 57]))
(array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16,
       17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33,
       34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50,
       51, 52, 53, 54, 55, 56, 57]), array([58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74,
       75, 76]))
(array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16,
       17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33,
       34, 35, 36, 37, 38, 39, 40

In [102]:
from sklearn.model_selection import TimeSeriesSplit
train_data = data.dropna()
precision1 = 0
train_len = 400
dataY = pd.DataFrame()
for seed in range(0, 7, 1):
    folds = TimeSeriesSplit(5)
    for skip in [30]:
        # 買いの場合　5日、50日
        # 売りの場合　3%
        dic = {}
        Ys = []
        for Date in ['2012-12-01']:#, '2015-06-01', '2015-09-01', '2015-12-01']:
            train_data['class'] = (train_data['現物'].pct_change(periods=skip) > train_data['現物'].pct_change(periods=skip).describe()['50%']).astype(int)
            #train_data=data.dropna(subset=['BPS指数ベース','利回り指数ベース', 'SCFI', '売り残', '買い残'])

            df = train_data[::]
            train_len = df.shape[0] // 2
            DATE = pd.to_datetime('2021-01-29') - relativedelta(weeks=skip)
            df = df.loc[DATE:, :]
            
            y = df.loc[DATE:, 'class'][skip:]
            
            # 欠損値を列の1つ手前の値で埋める
            %matplotlib inline


            df = df.fillna(method='ffill')

            

            #訓練データを説明変数データ(X_train)と目的変数データ(y_train)に分割

            for num1 in [0.002]:

                # Old 250日前比較の変化分、 300日の変化分、 350日の変化分、425日の変化分、450日の変化分 
                # ['Bond_400', 'Bond_225', 'Bond_250', 'Bond_125', 'Bond_350']
                # ['PER加重平均', 'PER指数ベース', 'PBR加重平均', 'PBR指数ベース', 'EPS加重平均', 'EPS指数ベース', 'BPS加重平均', 'BPS指数ベース']
                # 'PBR指数ベース', 'EPS指数ベース'
                A = df.drop(['class'], axis=1)[['PBR指数ベース', 'EPS指数ベース']].dropna().replace(-np.inf, 0).replace(np.inf, 0)
                # 'PER指数ベース', 'BPS指数ベース', '利回り指数ベース', 'jabond',  'SCFI', 'CCFI', 'バルチック海運指数 BADI'
                # 'BPS指数ベース', '利回り指数ベース', 'SCFI'
                B = df.drop(['class'], axis=1)[['BPS指数ベース','利回り指数ベース', 'SCFI', '売り残']].sample(2, random_state=seed, axis=1).dropna().replace(-np.inf, 0).replace(np.inf, 0)
                #X = pd.concat([A, B], axis=1).dropna().loc['2021-01-29':, :]
                
                cols =['先物合計','売り残', '売り残前回比', '買い残', '買い残前回比', '比率3', 'ラージ', 'ミニ',  'SCFI', 
                       'PER指数ベース1wChange', 'PBR指数ベース1wChange', '利回り指数ベース1wChange', 'EPS指数ベース1wChange','BPS指数ベース1wChange', '累積']
                
                X = df.loc['2021-01-29':, cols].replace(-np.inf, 0).replace(np.inf, 0).sample(12, random_state=seed, axis=1)
                
                
                
                
                
                train_len = 19


                rate_prediction = []
                
                #X = df.drop(['class'], axis=1)

                for i in tqdm(range(0,len(y)-train_len+1, 1)):
                    print(i)
                    try:
                        indexes = list(folds.split(X[:train_len+i].values))[4]
                        X_train, X_test, y_train, y_test = X[:train_len+i].values[indexes[0]], X[:train_len+i].values[indexes[1]], y[:train_len+i].values[indexes[0]], y[:train_len+i].values[indexes[1]]
                        #print(X_train, X_test, y_train, y_test)
                        X_test, X_valid, y_test, y_valid = X_test[-1], X_test[:-1] ,y_test[-1], y_test[:-1]
                    except Exception as e:
                        print(e, X_test)
                        continue
                    model = lstm_pred(X[:train_len+i].values)
                    # Winsorize top 1% and bottom 1% of points.
                    # Apply on X_train and X_test separately
                    '''
                    X_train = mstats.winsorize(X_train, limits = [0.01, 0.01])
                    X_predict = mstats.winsorize(X_predict, limits = [0.01, 0.01])
                    '''


                    model.fit(X_train,
                              y_train, 
                              epochs=30, 
                              batch_size=32, 
                              validation_data=(X_valid, y_valid),
                              callbacks=callbacks)

                    
                    # Make a prediction on testing data
                    # [[],[],[],,,] -> [ , , ,]
                    #plot_feature_importance(model.feature_importances_,X.columns,'CATBOOST')
                    try:
                        pred = model.predict(X_test.reshape(1, -1))
                    except Exception as e:
                        print(e)
                        continue

                    rate_prediction = np.append(rate_prediction, pred)
                    Ys.append(y_test)
                    precision = matthews_corrcoef(Ys, rate_prediction)
                    #print(precision)
                dic['No'] = seed
                dic2 = {name:[1] if name in X.columns else [0] for name in cols}
                dic.update(dic2)
                dic['precision'] = precision
                temp = pd.DataFrame(dic)
                try:
                    result = pd.DataFrame(rate_prediction, index=X.index[train_len+11:], columns=['predict'])
                except:
                    continue
                    #result = pd.DataFrame(rate_prediction, index=X.index[train_len-1:-1], columns=['predict'])
                precision = matthews_corrcoef(Ys[:], rate_prediction)
                if precision > precision1:
                    pair = ('corr:', precision, Date, X.columns, skip, seed)
                    precision1 = precision
                print('corr:', precision, Date, X.columns)
            dataY = pd.concat([dataY, temp])
            
            print(result.sum()['predict'])



result
display(dataY)

  0%|          | 0/67 [00:00<?, ?it/s]

0
Epoch 1/30
1/1 ━━━━━━━━━━━━━━━━━━━━ 2s 2s/step - loss: 0.4375 - val_loss: 0.5000
Epoch 2/30
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 85ms/step - loss: 0.4375 - val_loss: 0.5000
Epoch 3/30
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 84ms/step - loss: 0.4375 - val_loss: 0.5000
Epoch 4/30
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 82ms/step - loss: 0.4375 - val_loss: 0.5000
Epoch 5/30
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 84ms/step - loss: 0.4375 - val_loss: 0.5000
Epoch 5: early stopping
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 199ms/step
1
Epoch 1/30
1/1 ━━━━━━━━━━━━━━━━━━━━ 2s 2s/step - loss: 0.4706 - val_loss: 0.5000
Epoch 2/30
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 84ms/step - loss: 0.4706 - val_loss: 0.5000
Epoch 3/30
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 82ms/step - loss: 0.4706 - val_loss: 0.5000
Epoch 4/30
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 82ms/step - loss: 0.4706 - val_loss: 0.5000
Epoch 5/30
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 82ms/step - loss: 0.4706 - val_loss: 0.5000
Epoch 5: early stopping
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 196ms/step
2
Epoch 1/30
1/1 ━━━━━━━━━━━━━━━━━━━━ 3s 3s/

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 202ms/step
24
Epoch 1/30
2/2 ━━━━━━━━━━━━━━━━━━━━ 2s 309ms/step - loss: 0.4792 - val_loss: 0.1667
Epoch 2/30
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 40ms/step - loss: 0.4792 - val_loss: 0.1667
Epoch 3/30
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 40ms/step - loss: 0.4792 - val_loss: 0.1667
Epoch 4/30
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 42ms/step - loss: 0.4792 - val_loss: 0.1667
Epoch 5/30
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 40ms/step - loss: 0.4792 - val_loss: 0.1667
Epoch 5: early stopping
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 204ms/step
25
Epoch 1/30
2/2 ━━━━━━━━━━━━━━━━━━━━ 3s 320ms/step - loss: 0.4458 - val_loss: 0.1667
Epoch 2/30
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 46ms/step - loss: 0.4000 - val_loss: 0.1667
Epoch 3/30
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 43ms/step - loss: 0.4458 - val_loss: 0.1667
Epoch 4/30
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 43ms/step - loss: 0.4458 - val_loss: 0.1667
Epoch 5/30
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 50ms/step - loss: 0.4917 - val_loss: 0.1667
Epoch 5: early stopping
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 220ms/st

Epoch 5: early stopping
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 217ms/step
36
Epoch 1/30
2/2 ━━━━━━━━━━━━━━━━━━━━ 3s 309ms/step - loss: 0.4554 - val_loss: 0.3750
Epoch 2/30
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 45ms/step - loss: 0.4554 - val_loss: 0.3750
Epoch 3/30
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 54ms/step - loss: 0.4613 - val_loss: 0.3750
Epoch 4/30
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 41ms/step - loss: 0.4583 - val_loss: 0.3750
Epoch 5/30
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 43ms/step - loss: 0.4613 - val_loss: 0.3750
Epoch 5: early stopping
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 199ms/step
37
Epoch 1/30
2/2 ━━━━━━━━━━━━━━━━━━━━ 3s 306ms/step - loss: 0.4500 - val_loss: 0.3750
Epoch 2/30
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 43ms/step - loss: 0.4458 - val_loss: 0.3750
Epoch 3/30
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 43ms/step - loss: 0.4472 - val_loss: 0.3750
Epoch 4/30
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 46ms/step - loss: 0.4472 - val_loss: 0.3750
Epoch 5/30
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 47ms/step - loss: 0.4486 - val_loss: 0.3750
Epoch 5: early stopping
1/1 ━━━━━━━━

3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step - loss: 0.5391 - val_loss: 0.6667
Epoch 5/30
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step - loss: 0.5156 - val_loss: 0.6667
Epoch 5: early stopping
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 208ms/step
60
Epoch 1/30
3/3 ━━━━━━━━━━━━━━━━━━━━ 3s 195ms/step - loss: 0.4792 - val_loss: 0.6667
Epoch 2/30
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step - loss: 0.5143 - val_loss: 0.6667
Epoch 3/30
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step - loss: 0.5299 - val_loss: 0.6667
Epoch 4/30
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step - loss: 0.4323 - val_loss: 0.6667
Epoch 5/30
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step - loss: 0.3581 - val_loss: 0.6667
Epoch 5: early stopping
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 224ms/step
61
Epoch 1/30
3/3 ━━━━━━━━━━━━━━━━━━━━ 3s 195ms/step - loss: 0.4601 - val_loss: 0.6667
Epoch 2/30
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 40ms/step - loss: 0.4371 - val_loss: 0.6667
Epoch 3/30
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step - loss: 0.4757 - val_loss: 0.6667
Epoch 4/30
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 36ms/ste

ValueError: Shape of passed values is (67, 1), indices imply (93, 1)

In [103]:
result

,predict
日付,
2022-01-07,1.0
2022-01-14,1.0
2022-01-21,1.0
2022-01-28,1.0
2022-02-18,1.0
...,...
2024-08-09,1.0
2024-08-23,1.0
2024-09-06,1.0


In [ ]:
'現物', '先物合計'
       '売り残', '売り残前回比', '買い残', '買い残前回比', '比率3', 'ラージ', 'ミニ',
       'SCFI', 'PER加重平均1wChange', 'PER指数ベース1wChange',
       'PBR加重平均1wChange', 'PBR指数ベース1wChange', '利回り単純計算1wChange',
       '利回り指数ベース1wChange', 'EPS加重平均1wChange', 'EPS指数ベース1wChange',
       'BPS加重平均1wChange', 'BPS指数ベース1wChange', '累積'

In [61]:
cols

Index(['PER加重平均', 'PER指数ベース', 'PBR加重平均', 'PBR指数ベース', '利回り単純計算', '利回り指数ベース',
       'EPS加重平均', 'EPS指数ベース', 'BPS加重平均', 'BPS指数ベース', '現物', '先物合計', '現物先物合計',
       '売り残', '売り残前回比', '買い残', '買い残前回比', '差引', '比率', '比率2', '比率3', 'ラージ', 'ミニ',
       'TOPIX', 'Date', 'SCFI', 'PER加重平均1wChange', 'PER指数ベース1wChange',
       'PBR加重平均1wChange', 'PBR指数ベース1wChange', '利回り単純計算1wChange',
       '利回り指数ベース1wChange', 'EPS加重平均1wChange', 'EPS指数ベース1wChange',
       'BPS加重平均1wChange', 'BPS指数ベース1wChange', '累積', 'class'],
      dtype='object')

In [74]:
pd.DataFrame(rate_prediction, index=X.index[train_len+11:], columns=['predict']).sum()

predict    82.0
dtype: float64

In [80]:
X.dropna().shape

(112, 15)